In [1]:
import math
from datetime import timedelta

import numpy as np
import pandas as pd

df = pd.read_csv("model_stats.csv")
df.head()

,model_name,dataset_name,job_type,group,n_gpus,runtime,n_samples,epochs,max_epochs
0,microsoft/beit-large-patch16-512,aharley/rvl_cdip,train,aharley/rvl_cdip_20231130-151813-370445,1,254866.072681,896000.0,2.8,10.0
1,facebook/deit-small-patch16-224,aharley/rvl_cdip,train,aharley/rvl_cdip_20231130-151813-370445,2,86460.060732,3200000.0,10.0,10.0
2,microsoft/beit-large-patch16-512,huggan/wikiart,train,huggan/wikiart_20231130-184447-563098,1,172383.368446,488664.0,8.0,10.0
3,facebook/dinov2-base-imagenet1k-1-layer,pcuenq/oxford-pets,train,pcuenq/oxford-pets_20231130-134100-654677,1,2775.050471,55117.5,10.0,10.0
4,facebook/dinov2-small-imagenet1k-1-layer,pcuenq/oxford-pets,train,pcuenq/oxford-pets_20231130-134100-654677,1,2134.253396,55117.5,10.0,10.0


In [2]:
df["samples_per_gpu_per_s"] = df["n_samples"] / (df["n_gpus"] * df["runtime"])
df_train = df[df["job_type"] == "train"]
df_train = df_train[df_train["max_epochs"] == 10]
df_train = df_train[df_train["epochs"] >= 1]

df_metrics = df[df["job_type"] == "metrics"]
df_metrics = df_metrics[
    ~(
        (df_metrics["dataset_name"] == "pcuenq/oxford-pets")
        & (df_metrics["n_samples"] > 8000)
    )
]

In [3]:
means = (
    df_train[df_train["n_gpus"] == 1]
    .groupby("model_name")["samples_per_gpu_per_s"]
    .mean()
)
means.to_csv("mean_train.csv")
means

model_name
Intel/vit-base-patch16-224-int8-static            22.579535
apple/mobilevit-small                             23.563598
apple/mobilevit-xx-small                          23.858503
facebook/deit-base-patch16-224                    22.762595
facebook/deit-base-patch16-384                    10.942464
facebook/deit-small-patch16-224                   28.907155
facebook/deit-tiny-patch16-224                    30.221383
facebook/dinov2-base-imagenet1k-1-layer           20.021727
facebook/dinov2-small-imagenet1k-1-layer          25.685017
google/vit-base-patch16-384                       10.926187
google/vit-large-patch16-224                      13.333739
microsoft/beit-base-patch16-224                   22.444781
microsoft/beit-base-patch16-224-pt22k-ft22k       21.965131
microsoft/beit-large-patch16-512                   3.099107
microsoft/cvt-13                                  23.468707
microsoft/cvt-21-384-22k                          12.485401
microsoft/swin-base-patch4-wi

In [4]:
(means / means.median()).round(1).sort_values()

model_name
microsoft/beit-large-patch16-512                  0.1
microsoft/swin-large-patch4-window12-384-in22k    0.3
google/vit-base-patch16-384                       0.5
microsoft/swin-base-patch4-window12-384           0.5
facebook/deit-base-patch16-384                    0.5
microsoft/cvt-21-384-22k                          0.6
google/vit-large-patch16-224                      0.6
microsoft/swinv2-base-patch4-window8-256          0.7
facebook/dinov2-base-imagenet1k-1-layer           0.9
microsoft/beit-base-patch16-224                   1.0
microsoft/beit-base-patch16-224-pt22k-ft22k       1.0
facebook/deit-base-patch16-224                    1.0
Intel/vit-base-patch16-224-int8-static            1.0
microsoft/cvt-13                                  1.1
apple/mobilevit-xx-small                          1.1
apple/mobilevit-small                             1.1
facebook/dinov2-small-imagenet1k-1-layer          1.2
facebook/deit-small-patch16-224                   1.3
facebook/deit-tin

In [5]:
mins = df_train.groupby("model_name")["samples_per_gpu_per_s"].min().sort_values()
mins.to_csv("mins_train.csv")

In [6]:
(mins / mins.median()).round(1).sort_values()

model_name
microsoft/beit-large-patch16-512                  0.1
microsoft/swin-large-patch4-window12-384-in22k    0.2
microsoft/swin-base-patch4-window12-384           0.3
microsoft/cvt-21-384-22k                          0.3
google/vit-base-patch16-384                       0.3
facebook/deit-base-patch16-384                    0.3
google/vit-large-patch16-224                      0.6
microsoft/swinv2-base-patch4-window8-256          0.8
facebook/deit-base-patch16-224                    1.0
apple/mobilevit-small                             1.0
facebook/dinov2-base-imagenet1k-1-layer           1.0
facebook/deit-small-patch16-224                   1.0
Intel/vit-base-patch16-224-int8-static            1.1
microsoft/beit-base-patch16-224                   1.1
microsoft/beit-base-patch16-224-pt22k-ft22k       1.1
microsoft/cvt-13                                  1.1
apple/mobilevit-xx-small                          1.1
facebook/dinov2-small-imagenet1k-1-layer          1.3
facebook/deit-tin

In [7]:
mean_metrics = (
    df_metrics.groupby("model_name")["samples_per_gpu_per_s"].mean().sort_values()
)
mean_metrics.to_csv("mean_metrics.csv")
mean_metrics

model_name
microsoft/beit-large-patch16-512                   8.709086
microsoft/swin-large-patch4-window12-384-in22k    16.878391
facebook/deit-base-distilled-patch16-224          18.019147
facebook/deit-base-distilled-patch16-384          18.135935
google/vit-large-patch16-224                      21.633657
microsoft/swin-base-patch4-window12-384           21.754460
google/vit-base-patch16-384                       22.622347
facebook/deit-base-patch16-384                    22.882780
microsoft/cvt-21-384-22k                          23.449650
microsoft/swinv2-base-patch4-window8-256          25.016573
facebook/dinov2-base-imagenet1k-1-layer           25.448208
microsoft/cvt-13                                  29.356442
microsoft/beit-base-patch16-224                   30.749740
facebook/dinov2-small-imagenet1k-1-layer          30.892294
apple/mobilevit-small                             31.135313
microsoft/beit-base-patch16-224-pt22k-ft22k       31.676883
facebook/levit-128S          

In [8]:
min_metrics = (
    df_metrics.groupby("model_name")["samples_per_gpu_per_s"].min().sort_values()
)
min_metrics.to_csv("mins_metrics.csv")
min_metrics

model_name
microsoft/beit-large-patch16-512                   1.378881
microsoft/swin-large-patch4-window12-384-in22k     4.036574
google/vit-base-patch16-384                        5.375579
google/vit-large-patch16-224                       6.016419
facebook/deit-tiny-patch16-224                     6.116535
facebook/deit-base-distilled-patch16-224           6.328108
facebook/deit-base-distilled-patch16-384           6.774597
facebook/deit-base-patch16-384                     6.894014
microsoft/swin-base-patch4-window12-384            7.854860
microsoft/cvt-21-384-22k                          10.077953
facebook/dinov2-small-imagenet1k-1-layer          10.433780
facebook/dinov2-base-imagenet1k-1-layer           10.496028
facebook/deit-small-patch16-224                   10.881766
microsoft/swinv2-base-patch4-window8-256          11.084654
facebook/deit-base-patch16-224                    11.090990
Intel/vit-base-patch16-224-int8-static            11.301282
microsoft/beit-base-patch16-2

In [9]:
df_metrics[df_metrics["model_name"] == "microsoft/beit-large-patch16-512"]

,model_name,dataset_name,job_type,group,n_gpus,runtime,n_samples,epochs,max_epochs,samples_per_gpu_per_s
55,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,640.874717,7390.0,NaN,NaN,11.531115
57,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,468.329532,5500.0,NaN,NaN,11.743868
58,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,280.416340,3500.0,NaN,NaN,12.481441
60,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,197.095700,2000.0,NaN,NaN,10.147355
62,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,62.312085,500.0,NaN,NaN,8.024126
64,microsoft/beit-large-patch16-512,pcuenq/oxford-pets,metrics,pcuenq/oxford-pets_20231121-190822-362963,1,103.395887,1000.0,NaN,NaN,9.671565
200,microsoft/beit-large-patch16-512,huggan/wikiart,metrics,huggan/wikiart_20231117-102011-747776,1,3345.674565,30000.0,NaN,NaN,8.966802
201,microsoft/beit-large-patch16-512,huggan/wikiart,metrics,huggan/wikiart_20231117-102011-747776,1,6595.611371,60000.0,NaN,NaN,9.096958
202,microsoft/beit-large-patch16-512,huggan/wikiart,metrics,huggan/wikiart_20231117-102011-747776,1,1171.557837,10000.0,NaN,NaN,8.535643
203,microsoft/beit-large-patch16-512,huggan/wikiart,metrics,huggan/wikiart_20231117-102011-747776,1,131.759124,1000.0,NaN,NaN,7.589607


In [10]:
27971.468225 / 60 / 60

7.769852284722222

In [37]:
tmp = (
    df_train[(df_train["dataset_name"] == "aharley/rvl_cdip")]
    .groupby(["model_name", "n_gpus", "epochs"])["samples_per_gpu_per_s"]
    .min()
)
tmp

model_name                        n_gpus  epochs
facebook/deit-small-patch16-224   2       10.0      18.505654
microsoft/beit-large-patch16-512  1       2.8        3.515572
Name: samples_per_gpu_per_s, dtype: float64

In [38]:
tmp.to_csv("rvl_cdip_mins_epochs.csv")

In [42]:
pets = pd.read_csv("pets_mins.csv", index_col="model_name")
wikiart = pd.read_csv("wikiart_mins.csv", index_col="model_name")

In [43]:
wikiart

,n_gpus,samples_per_gpu_per_s
model_name,,
Intel/vit-base-patch16-224-int8-static,1,24.662853
apple/mobilevit-small,1,25.676141
apple/mobilevit-xx-small,1,23.916375
facebook/deit-base-patch16-384,2,6.552416
facebook/dinov2-base-imagenet1k-1-layer,1,19.803804
facebook/dinov2-small-imagenet1k-1-layer,1,24.999745
google/vit-base-patch16-384,2,5.924763
google/vit-large-patch16-224,1,13.712902
microsoft/beit-base-patch16-224,1,23.591532


In [44]:
pets

,n_gpus,samples_per_gpu_per_s
model_name,,
Intel/vit-base-patch16-224-int8-static,1,20.496217
apple/mobilevit-small,1,21.832720
apple/mobilevit-xx-small,1,21.763626
facebook/deit-base-patch16-224,1,20.397917
facebook/deit-base-patch16-384,1,10.638746
facebook/deit-small-patch16-224,1,28.021551
facebook/deit-tiny-patch16-224,1,29.764743
facebook/dinov2-base-imagenet1k-1-layer,1,19.462187
facebook/dinov2-small-imagenet1k-1-layer,1,25.825190


In [49]:
pets["hours_500000_10epochs"] = (10 * 50000 / pets["samples_per_gpu_per_s"]) / (60 * 60)
pets.sort_values(by="hours_500000_10epochs")

,n_gpus,samples_per_gpu_per_s,hours_50k_10epochs
model_name,,,
facebook/deit-tiny-patch16-224,1,29.764743,4.666222
facebook/deit-small-patch16-224,1,28.021551,4.956502
facebook/dinov2-small-imagenet1k-1-layer,1,25.825190,5.378039
apple/mobilevit-small,1,21.832720,6.361502
apple/mobilevit-xx-small,1,21.763626,6.381698
microsoft/cvt-13,1,21.476572,6.466995
microsoft/beit-base-patch16-224-pt22k-ft22k,1,20.567659,6.752781
microsoft/beit-base-patch16-224,1,20.500991,6.774740
Intel/vit-base-patch16-224-int8-static,1,20.496217,6.776318


In [53]:
pets.sort_values(by="hours_500000_10epochs", inplace=True)

In [54]:
pets

,samples_per_gpu_per_s,hours_50k_10epochs
model_name,,
facebook/deit-tiny-patch16-224,29.764743,4.666222
facebook/deit-small-patch16-224,28.021551,4.956502
facebook/dinov2-small-imagenet1k-1-layer,25.825190,5.378039
apple/mobilevit-small,21.832720,6.361502
apple/mobilevit-xx-small,21.763626,6.381698
microsoft/cvt-13,21.476572,6.466995
microsoft/beit-base-patch16-224-pt22k-ft22k,20.567659,6.752781
microsoft/beit-base-patch16-224,20.500991,6.774740
Intel/vit-base-patch16-224-int8-static,20.496217,6.776318


In [59]:
group_small = pets[pets["hours_500000_10epochs"] < 8].index.values
group_small

array(['facebook/deit-tiny-patch16-224',
       'facebook/deit-small-patch16-224',
       'facebook/dinov2-small-imagenet1k-1-layer',
       'apple/mobilevit-small', 'apple/mobilevit-xx-small',
       'microsoft/cvt-13', 'microsoft/beit-base-patch16-224-pt22k-ft22k',
       'microsoft/beit-base-patch16-224',
       'Intel/vit-base-patch16-224-int8-static',
       'facebook/deit-base-patch16-224',
       'facebook/dinov2-base-imagenet1k-1-layer'], dtype=object)

In [61]:
group_medium = pets[
    (pets["hours_500000_10epochs"] > 8) * (pets["hours_500000_10epochs"] < 16)
].index.values
group_medium

array(['microsoft/swinv2-base-patch4-window8-256',
       'google/vit-large-patch16-224', 'microsoft/cvt-21-384-22k',
       'facebook/deit-base-patch16-384', 'google/vit-base-patch16-384',
       'microsoft/swin-base-patch4-window12-384'], dtype=object)

In [62]:
group_large = pets[(pets["hours_500000_10epochs"] > 16)].index.values
group_large

array(['microsoft/swin-large-patch4-window12-384-in22k',
       'microsoft/beit-large-patch16-512'], dtype=object)

In [73]:
def get_group(row):
    name = row.name
    if name in group_small:
        return "small"
    if name in group_medium:
        return "medium"
    if name in group_large:
        return "large"


pets["group"] = pets.apply(get_group, axis=1)

In [85]:
safety_factor = 1.5
hours_small = math.ceil(8 * safety_factor)
hours_medium = math.ceil(15 * safety_factor)
hours_large = math.ceil(48 * safety_factor)
hours_medium

23

In [86]:
def get_50khours(row):
    if row.group == "small":
        return hours_small
    if row.group == "medium":
        return hours_medium
    if row.group == "large":
        return hours_large


pets["request_hours_500000"] = pets.apply(get_50khours, axis=1)
pets

,samples_per_gpu_per_s,hours_50k_10epochs,group,request_hours_50k
model_name,,,,
facebook/deit-tiny-patch16-224,29.764743,4.666222,small,12
facebook/deit-small-patch16-224,28.021551,4.956502,small,12
facebook/dinov2-small-imagenet1k-1-layer,25.825190,5.378039,small,12
apple/mobilevit-small,21.832720,6.361502,small,12
apple/mobilevit-xx-small,21.763626,6.381698,small,12
microsoft/cvt-13,21.476572,6.466995,small,12
microsoft/beit-base-patch16-224-pt22k-ft22k,20.567659,6.752781,small,12
microsoft/beit-base-patch16-224,20.500991,6.774740,small,12
Intel/vit-base-patch16-224-int8-static,20.496217,6.776318,small,12


In [87]:
pets["safety_margin_500000"] = (
    pets["request_hours_500000"] / pets["hours_500000_10epochs"]
)
pets

,samples_per_gpu_per_s,hours_50k_10epochs,group,request_hours_50k,safety_margin_50k
model_name,,,,,
facebook/deit-tiny-patch16-224,29.764743,4.666222,small,12,2.571674
facebook/deit-small-patch16-224,28.021551,4.956502,small,12,2.421062
facebook/dinov2-small-imagenet1k-1-layer,25.825190,5.378039,small,12,2.231296
apple/mobilevit-small,21.832720,6.361502,small,12,1.886347
apple/mobilevit-xx-small,21.763626,6.381698,small,12,1.880377
microsoft/cvt-13,21.476572,6.466995,small,12,1.855576
microsoft/beit-base-patch16-224-pt22k-ft22k,20.567659,6.752781,small,12,1.777046
microsoft/beit-base-patch16-224,20.500991,6.774740,small,12,1.771286
Intel/vit-base-patch16-224-int8-static,20.496217,6.776318,small,12,1.770873


In [96]:
samples = [500, 1000, 5000, 10000]
for s in samples:
    pets[f"request_hours_{s}"] = (
        np.ceil(pets["request_hours_50k"] * (s / 50000))
    ).astype(int)

In [146]:
pets.to_csv("model_groups_train.csv")

In [145]:
for samples in [500, 1000, 5000, 10000, 50000]:
    walltime = []
    for hours in pets[f"request_hours_{samples}"]:
        td = timedelta(hours=hours)
        walltime.append(f"{td.days}-{int(td.seconds / (60 * 60))}:0:0")
    pets[f"request_walltime_{samples}"] = walltime


pets

,samples_per_gpu_per_s,hours_50000_10epochs,group,request_hours_50000,safety_margin_50000,request_hours_500,request_hours_1000,request_hours_5000,request_hours_10000,request_walltime_500,request_walltime_1000,request_walltime_5000,request_walltime_10000,request_walltime_50000
model_name,,,,,,,,,,,,,,
facebook/deit-tiny-patch16-224,29.764743,4.666222,small,12,2.571674,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
facebook/deit-small-patch16-224,28.021551,4.956502,small,12,2.421062,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
facebook/dinov2-small-imagenet1k-1-layer,25.825190,5.378039,small,12,2.231296,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
apple/mobilevit-small,21.832720,6.361502,small,12,1.886347,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
apple/mobilevit-xx-small,21.763626,6.381698,small,12,1.880377,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/cvt-13,21.476572,6.466995,small,12,1.855576,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/beit-base-patch16-224-pt22k-ft22k,20.567659,6.752781,small,12,1.777046,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/beit-base-patch16-224,20.500991,6.774740,small,12,1.771286,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
Intel/vit-base-patch16-224-int8-static,20.496217,6.776318,small,12,1.770873,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0


In [104]:
min_metrics

model_name
microsoft/beit-large-patch16-512                   1.378881
microsoft/swin-large-patch4-window12-384-in22k     4.036574
google/vit-base-patch16-384                        5.375579
google/vit-large-patch16-224                       6.016419
facebook/deit-tiny-patch16-224                     6.116535
facebook/deit-base-distilled-patch16-224           6.328108
facebook/deit-base-distilled-patch16-384           6.774597
facebook/deit-base-patch16-384                     6.894014
microsoft/swin-base-patch4-window12-384            7.854860
microsoft/cvt-21-384-22k                          10.077953
facebook/dinov2-small-imagenet1k-1-layer          10.433780
facebook/dinov2-base-imagenet1k-1-layer           10.496028
facebook/deit-small-patch16-224                   10.881766
microsoft/swinv2-base-patch4-window8-256          11.084654
facebook/deit-base-patch16-224                    11.090990
Intel/vit-base-patch16-224-int8-static            11.301282
microsoft/beit-base-patch16-2

In [105]:
mean_metrics

model_name
microsoft/beit-large-patch16-512                   8.709086
microsoft/swin-large-patch4-window12-384-in22k    16.878391
facebook/deit-base-distilled-patch16-224          18.019147
facebook/deit-base-distilled-patch16-384          18.135935
google/vit-large-patch16-224                      21.633657
microsoft/swin-base-patch4-window12-384           21.754460
google/vit-base-patch16-384                       22.622347
facebook/deit-base-patch16-384                    22.882780
microsoft/cvt-21-384-22k                          23.449650
microsoft/swinv2-base-patch4-window8-256          25.016573
facebook/dinov2-base-imagenet1k-1-layer           25.448208
microsoft/cvt-13                                  29.356442
microsoft/beit-base-patch16-224                   30.749740
facebook/dinov2-small-imagenet1k-1-layer          30.892294
apple/mobilevit-small                             31.135313
microsoft/beit-base-patch16-224-pt22k-ft22k       31.676883
facebook/levit-128S          

In [107]:
df_metrics.groupby("n_samples")["runtime"].max()

n_samples
500.0         362.612993
1000.0        640.629446
2000.0        197.095700
3500.0        280.416340
5500.0        468.329532
7390.0        640.874717
10000.0      1171.557837
30000.0      3345.674565
60000.0      6595.611371
300000.0    27971.468225
Name: runtime, dtype: float64

In [117]:
request_metrics = np.ceil(
    df_metrics.groupby("n_samples")["runtime"].max() * safety_factor / (60 * 60)
).astype(int)

In [118]:
request_metrics

n_samples
500.0        1
1000.0       1
2000.0       1
3500.0       1
5500.0       1
7390.0       1
10000.0      1
30000.0      2
60000.0      3
300000.0    12
Name: runtime, dtype: int64

In [120]:
request_metrics[5000] = 1
request_metrics[50000] = 3
request_metrics

n_samples
500.0        1
1000.0       1
2000.0       1
3500.0       1
5500.0       1
7390.0       1
10000.0      1
30000.0      2
60000.0      3
300000.0    12
5000.0       1
50000.0      3
Name: runtime, dtype: int64

In [123]:
subset_requests = request_metrics[[500, 1000, 5000, 10000, 50000]]
subset_requests.index = subset_requests.index.astype(int)
subset_requests.name = "request_hours"
subset_requests

n_samples
500      1
1000     1
5000     1
10000    1
50000    3
Name: request_hours, dtype: int64

In [124]:
subset_requests.to_csv("metrics_request.csv")

In [128]:
df_subset_requests = pd.DataFrame(subset_requests)
df_subset_requests

In [135]:
df_subset_requests
df_walltime = [f"0-{hours}:0:0" for hours in df_subset_requests["request_hours"]]
df_subset_requests["walltime"] = df_walltime

In [149]:
df_subset_requests.to_csv("metrics_request.csv")
df_subset_requests

,request_hours,walltime
n_samples,,
500,1,0-1:0:0
1000,1,0-1:0:0
5000,1,0-1:0:0
10000,1,0-1:0:0
50000,3,0-3:0:0


In [148]:
df_subset_requests

,request_hours,walltime
n_samples,,
500,1,0-1:0:0
1000,1,0-1:0:0
5000,1,0-1:0:0
10000,1,0-1:0:0
50000,3,0-3:0:0


In [153]:
pets["request_hours_50000"].max()

72

In [154]:
50000 / 0.75

66666.66666666667

In [155]:
0.75 * 66666

49999.5

In [156]:
wikiart_full = 81444

In [159]:
wikiart_val = wikiart_full * 0.1
wikiart_test = wikiart_full * 0.25
print(wikiart_val, wikiart_test)

8144.400000000001 20361.0


In [160]:
pets_full = 7390
pets_train = pets_full * 0.75
pets_val = pets_full * 0.25

In [171]:
epochs = 10
n_val = 50
pets_samples = (
    2 * epochs * pets_train + n_val * pets_val
)  # * 2 train to account for back prop?
pets_samples

203225.0

In [162]:
50000 * epochs + wikiart_val * 50 + wikiart_test

927581.0

In [163]:
(
    50000 * epochs + 40000 * epochs + 40000
)  # rvl cdip 50000 train, 40000 val once per epoch, 40000 test

940000

In [170]:
pets_max = 18288.377133

pets_max / (60 * 60)

5.080104759166667

In [172]:
def get_samples(train_size, val_size, test_size, epochs, n_val):
    # * 2 fudge factor for train samples to account for back prop?
    return 2 * train_size * epochs + val_size * n_val + test_size


def get_time(samples, ref_samples=pets_samples, ref_time=pets_max):
    return ref_time * (samples / ref_samples)

In [176]:
s = get_samples(500, 500, 40000, 10, 50)
t = get_time(s) / (60 * 60)
print(s, t)

75000 1.8748080055972445


In [177]:
pets["request_hours_500"].max()

1

In [180]:
df_metrics["runtime"].max() / (60 * 60)

7.769852284722858

In [185]:
df_walltime

['0-1:0:0', '0-1:0:0', '0-1:0:0', '0-1:0:0', '0-3:0:0']

In [186]:
# group, walltime_500, walltime_1000, walltime_5000, walltime_10000, walltime_50000
# small, "0-1:0:0", "0-1:0:0", "0-2:0:0", "0-3:0:0", "0-12:0:0"
# medium, "0-1:0:0", "0-1:0:0", "0-3:0:0", "0-5:0:0", "0-23:0:0"
# large, "0-1:0:0", "0-2:0:0", "0-8:0:0", "0-15:0:0", "3-0:0:0"

,samples_per_gpu_per_s,hours_50000_10epochs,group,request_hours_50000,safety_margin_50000,request_hours_500,request_hours_1000,request_hours_5000,request_hours_10000,request_walltime_500,request_walltime_1000,request_walltime_5000,request_walltime_10000,request_walltime_50000
model_name,,,,,,,,,,,,,,
facebook/deit-tiny-patch16-224,29.764743,4.666222,small,12,2.571674,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
facebook/deit-small-patch16-224,28.021551,4.956502,small,12,2.421062,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
facebook/dinov2-small-imagenet1k-1-layer,25.825190,5.378039,small,12,2.231296,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
apple/mobilevit-small,21.832720,6.361502,small,12,1.886347,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
apple/mobilevit-xx-small,21.763626,6.381698,small,12,1.880377,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/cvt-13,21.476572,6.466995,small,12,1.855576,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/beit-base-patch16-224-pt22k-ft22k,20.567659,6.752781,small,12,1.777046,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
microsoft/beit-base-patch16-224,20.500991,6.774740,small,12,1.771286,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
Intel/vit-base-patch16-224-int8-static,20.496217,6.776318,small,12,1.770873,1,1,2,3,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0


In [187]:
pd.read_csv("train_groups.csv")

,group,walltime_500,walltime_1000,walltime_5000,walltime_10000,walltime_50000
0,small,0-1:0:0,0-1:0:0,0-2:0:0,0-3:0:0,0-12:0:0
1,medium,0-1:0:0,0-1:0:0,0-3:0:0,0-5:0:0,0-23:0:0
2,large,0-1:0:0,0-2:0:0,0-8:0:0,0-15:0:0,3-0:0:0


In [191]:
pd.read_csv("model_groups.csv")[["model_name", "group"]].to_csv(
    "model_groups_only.csv", index=False
)